In [1]:
coupon_list_train = pd.read_csv('../../team1_coupon/dev/data/coupon_list_train.csv')
coupon_list_test = pd.read_csv('../../team1_coupon/dev/data/coupon_list_test.csv')
user_list = pd.read_csv('../../team1_coupon/dev/data/user_list.csv')
coupon_purchases_train = pd.read_csv("../../team1_coupon/dev/data/coupon_detail_train.csv")

In [2]:
### merge to obtain (USER_ID) <-> (COUPON_ID with features) training set
purchased_coupons_train = coupon_purchases_train.merge(coupon_list_train, on='COUPON_ID_hash', how='inner')
purchased_coupons_train = purchased_coupons_train.merge(user_list, on = 'USER_ID_hash', how = 'outer')

In [3]:
### filter redundant features
features = ['COUPON_ID_hash', 'USER_ID_hash',
            'GENRE_NAME', 'DISCOUNT_PRICE', 'PRICE_RATE',
            'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
            'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
            'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']
purchased_coupons_train = purchased_coupons_train[features]

In [4]:
### create 'dummyuser' records in order to merge training and testing sets in one
coupon_list_test['USER_ID_hash'] = 'dummyuser'
### filter testing set consistently with training set
coupon_list_test = coupon_list_test[features]

In [5]:
### merge sets together
combined = pd.concat([purchased_coupons_train, coupon_list_test], axis=0)
### create two new features
combined['DISCOUNT_PRICE'] = 1 / np.log10(combined['DISCOUNT_PRICE'])
combined['PRICE_RATE'] = (combined['PRICE_RATE'] / 100) ** 2
features.extend(['DISCOUNT_PRICE', 'PRICE_RATE'])

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log10
  after removing the cwd from sys.path.


In [6]:
### convert categoricals to OneHotEncoder form
categoricals = ['GENRE_NAME', 'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED',
                'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN',
                'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']
combined_categoricals = combined[categoricals]
combined_categoricals = pd.get_dummies(combined_categoricals,
                                    dummy_na=False)

In [7]:
### leaving continuous features as is, obtain transformed dataset
continuous = list(set(features) - set(categoricals))
combined = pd.concat([combined[continuous], combined_categoricals], axis=1)
### remove NaN values
NAN_SUBSTITUTION_VALUE = 1
combined = combined.fillna(NAN_SUBSTITUTION_VALUE)
### split back into training and testing sets
train = combined[combined['USER_ID_hash'] != 'dummyuser']
test = combined[combined['USER_ID_hash'] == 'dummyuser']
test.drop('USER_ID_hash', inplace=True, axis=1)

/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [8]:
### find most appropriate coupon for every user (mean of all purchased coupons), in other words, user profile
train_dropped_coupons = train.drop('COUPON_ID_hash', axis=1)
user_profiles = train_dropped_coupons.groupby(by='USER_ID_hash').mean()
test_only_features = test.drop('COUPON_ID_hash', axis=1)

In [9]:
from scipy import spatial
def cosine_similarity(vector_1, vector_2):
    
    idx = vector_1.nonzero()[0]
    if len(idx) == 0:
        return 0
    vector_1, vector_2 = np.array(vector_1)[idx], np.array(vector_2)[idx]
    
    idx = vector_2.nonzero()[0]
    if len(idx) == 0:
        return 0
    vector_1, vector_2 = np.array(vector_1)[idx], np.array(vector_2)[idx]
    #print(vector_1, vector_2)
    return 1 - spatial.distance.cosine(vector_1, vector_2) 

In [10]:
%%time
matrix = []
for idx1 in range(len(user_profiles)):
    row = []
    for idx2 in range(len(test_only_features)):
        row.append(cosine_similarity(user_profiles.iloc[idx1], test_only_features.iloc[idx2]))
    matrix.append(row)

CPU times: user 40min 55s, sys: 11.5 s, total: 41min 6s
Wall time: 41min 51s


In [15]:
coupons_ids = test['COUPON_ID_hash']
result_index = user_profiles.index
result_columns = [coupons_ids[i] for i in range(0, test_only_features.T.shape[1])]
result_df = pd.DataFrame(index= result_index, columns=result_columns,
                      data=matrix)

In [16]:
submission = pd.DataFrame(columns=['USER_ID_hash','PURCHASED_COUPONS'])
submission['USER_ID_hash'] = result_df.index
submission['PURCHASED_COUPONS'] = result_df.T.apply(lambda x: " ".join(x.sort_values(ascending=False)[:21].index)).values

In [17]:
from datetime import datetime
now = datetime.now()
cur_time = now.strftime('%Y-%m-%d %H-%M-%S')

submission.to_csv('cosine'+cur_time+'.csv',index = False,header=True)